# Homework 1 - BUAN675B
Put your code and result under the space below each question


## Description
You are provided with the locations and numbers of potential customers for a set of 
cities. You task is to find the optimal warehouse location. The data can be accessed on 
Blackboard.
![hw1_pic.PNG](attachment:hw1_pic.PNG)

## Import libraries

In [16]:
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 75.5 MB/s eta 0:00:00


In [1]:
# Import any package you feel helpful.
import numpy as np
import pandas as pd
from mip import Model, minimize, BINARY, INTEGER, CONTINUOUS, xsum

## Problem 1
1. Use the center of gravity (COG) method to obtain the location of the warehouse.

In [18]:
# read in the data first
cities=pd.read_csv("/content/baun675B_HW1_data.csv")

In [19]:
cities.head()

,x,y,population,name
0,-125.054795,13.703696,42760,A
1,41.839656,-59.181868,7950,B
2,20.940089,5.932856,21905,C
3,-88.803093,-3.120039,36835,D
4,59.086009,-5.155562,38640,E


In [25]:


D = []
for i in range(len(cities)):
    dx = cities['x'] - cities.iloc[i]['x']
    dy = cities['y'] - cities.iloc[i]['y']
    d = (dx**2 + dy**2)**0.5
    D.append(d)

D = np.array(D)
nCities = len(cities)
V = set(range(nCities))

m = Model("TSP")

y = [m.add_var(var_type=INTEGER, lb=0) for i in V]
x = [[m.add_var(var_type=BINARY) for i in V] for j in V]

m.objective = minimize(xsum(D[i][j]*x[i][j] for i in V for j in V))

# leave each city once
for i in V:
    m += xsum(x[i][j] for j in V - {i}) == 1

# enter each city once
for j in V:
    m += xsum(x[i][j] for i in V - {j}) == 1

for i in V - {0}:
    for j in V - {0}:
        if i != j:
            m += y[j] - (nCities+1) * x[i][j] >= y[i] - nCities

warehouse = m.add_var(var_type=INTEGER, lb=0)
m += warehouse == 0

m.optimize()
warehouse_city = None
for i in V:
    for j in V:
        if x[i][j].x != 0 and j == 0:
            warehouse_city = i
            break
    if warehouse_city is not None:
        break

if warehouse_city is not None:
    warehouse_location = (cities.iloc[warehouse_city]['x'], cities.iloc[warehouse_city]['y'])
    print("Location of Warehouse:", warehouse_location)



Location of Warehouse: (-162.16969445807212, 8.1575503917187)


## Problem 2
2. Assuming these cities are also candidate locations for a warehouse, find the city that optimizes average distance per customer.

In [17]:
nCities = len(cities)
V = set(range(nCities))
m = Model("TSP")

y = [m.add_var(var_type=INTEGER, lb=0) for i in V]
x = [[m.add_var(var_type=BINARY) for i in V] for j in V]
z = [m.add_var(var_type=BINARY) for i in V]

m.objective = minimize(
    xsum([D[i][j]*x[i][j] * cities.iloc[j]['population'] for i in V for j in V]) / 
    sum(cities['population'])
)


# leave each city once
for i in V:
    m += xsum([x[i][j] for j in V - {i}]) == 1

# enter each city once
for j in V:
    m += xsum([x[i][j] for i in V - {j}]) == 1

# choose exactly one city as the warehouse
m += xsum(z) == 1

# if a city is not the warehouse, it must be visited
for i in V:
    m += z[i] <= xsum([x[j][i] for j in V])

for i in V - {0}:
    for j in V - {0}:
        if i != j:
            m += y[j] - (nCities+1) * x[i][j] >= y[i] - nCities

m.optimize()

warehouse_city = None
for i in V:
    if z[i].x != 0:
        warehouse_city = i
        break

if warehouse_city is not None:
    warehouse_location = (cities.iloc[warehouse_city]['x'], cities.iloc[warehouse_city]['y'])
    print("Location of optmal city:", warehouse_location)
    print("city is ",cities.iloc[warehouse_city]['name'] )




Location of optmal city: (135.9969027810696, 23.266191228831275)
city is  L


## Problem 3 - Transportation problem
3. Given the same setup as the transportation problem in lecture 3 (Example 3). A new market with demand 200 is added to the supply network now, where transportation costs are 8, 6, and 3 from plant 1 to 3 to the market. Which of the transport routes would you choose to minimize the total cost?

In [18]:

model = Model()

In [19]:

costs = [
    [4, 5, 6, 8, 10,8],
    [6, 4, 3, 5, 8,6],
    [9, 7, 4, 2, 4,3]]

capacities = [500, 500, 500]
demands = [80, 270, 250, 160, 180,200]


In [20]:
n = len(capacities) # number of plants
m = len(demands) # number of customers

In [21]:
[[(i,j) for j in range(m)] for i in range(n)]

[[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5)],
 [(1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5)],
 [(2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5)]]

In [22]:
x = [[model.add_var(var_type=CONTINUOUS, lb=0) for j in range(m)] for i in range(n)]

In [23]:
len(x), len(x[0])

(3, 6)

In [24]:
total_cost = xsum([xsum([costs[i][j]*x[i][j] for j in range(m)]) for i in range(n)])

In [25]:
model.objective = minimize(total_cost)

In [26]:
# capacity constraints
for i in range(n):
    model += xsum([x[i][j] for j in range(m)]) <= capacities[i]

In [27]:
# demand constraints
for j in range(m):
    model += xsum([x[i][j] for i in range(n)]) == demands[j]

In [28]:
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [29]:
[[x[i][j].x for j in range(m)] for i in range(n)]

[[80.0, 60.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 210.0, 250.0, 0.0, 0.0, 40.0],
 [0.0, 0.0, 0.0, 160.0, 180.0, 160.0]]

In [30]:
model.objective_value

3970.0

In [31]:
for i in range(n):
    print('products transported from', i, 'is', 
          sum([x[i][j].x for j in range(m)]))

products transported from 0 is 140.0
products transported from 1 is 500.0
products transported from 2 is 500.0


In [32]:
for j in range(m):
    print('products transported to', j, 'is', 
          sum([x[i][j].x for i in range(n)]))

products transported to 0 is 80.0
products transported to 1 is 270.0
products transported to 2 is 250.0
products transported to 3 is 160.0
products transported to 4 is 180.0
products transported to 5 is 200.0


From the output, it looks like the transportation routes chosen to minimize the total cost are:

- 140 units of product are transported from plant 0 to market 0
- 500 units of product are transported from plant 1 to market 1
- 500 units of product are transported from plant 2 to market 2

This means that the minimum total cost is achieved by transporting the following quantities:

- 80 units from market 0
- 270 units from market 1
- 250 units from market 2
- 160 units from market 3
- 180 units from market 4
- 200 units from market 5

So, these are the transport routes that would minimize the total cost.